In [4]:
import cv2
import time
import FaceNormalizationUtils as faceutils
import pickle
import numpy as np
import os
from scipy import stats
# mode and so on
from collections import Counter

import matplotlib.pyplot as plt

#MTCNN face detector
from mtcnn.mtcnn import MTCNN
#deepface
from deepface import DeepFace
from deepface.commons import functions

from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def calc_embs(X, batch_size=2):
    norm_images = prewhiten(X)
    pd = []
    for start in range(0, len(norm_images), batch_size):
        pd.append(model.predict_on_batch(norm_images[start:start+batch_size]))
    return l2_normalize(np.concatenate(pd))

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

# Designed for UTKFace as filename contains biometric information
def TrainEmbedingsUTKFace(folders,outputfile):
    nimgs = 0
    Xorig = []
    X = []
    Gender = []
    Age = []
    Etnia = []
    
    for directory in folders:
        #print(directory)
        for path, subdirs, files in os.walk(directory):
            for name in files:
                #print(name)
                if name.endswith(".jpg") and nimgs < 2000:
                #if name.endswith(".png"): # and nimgs < 3000:
                    img_path = os.path.join(path, name)
                    #print(img_path)

                    image = cv2.imread(img_path)

                    if (type(image) is np.ndarray):
                        #print(img_path)
                        # Search face 
                        values = DetectLargestFaceEyesMTCNN(image)
                        if values is not None:
                            #print(nimgs)
                            face, eyes, shape = values

                            # draws container
                            [x, y, w, h] = face
                            if x > -1:
                                # Eyes
                                [lex, ley, rex, rey] = eyes
                                if lex > -1:
                                    
                                    B, G, R = cv2.split(image)

                                    # Normalize for Facenet
                                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Bnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Gnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Rnorm = normalizatorHS.normf_image
                                    NormBGRHS = cv2.merge((Bnorm, Gnorm, Rnorm))
                                    #cv2.imshow("Normalized", NormBGRHS)

                                    # Cropping from HS for facenet
                                    # Usa nyoki https://github.com/nyoki-mtl/keras-facenet
                                    NormBGR = NormBGRHS[35:115, 39:119, :]

                                    # SOft biometric data are extracted from filename
                                    if NormBGR is not None:
                                        nimgs = nimgs + 1
                                        #print(nimgs)

                                        fsub = name.find('_')
                                        sage = name[:fsub]
                                        #print(sage)
                                        sub = name[fsub + 1:]
                                        fsub = sub.find('_')
                                        sgender = sub[:fsub]
                                        sub = sub[fsub + 1:]
                                        fsub = sub.find('_')
                                        setnia = sub[:fsub]

                                        # print(name)
                                        # print(sage)
                                        # print(sgender)
                                        # print(setnia)
                                        # print(nimgs)

                                        # Facenet kearas expects 160x160
                                        #imaged = cv2.resize(NormBGR, (160, 160))
                                        
                                        # Obtiene embeddings
                                        imaged = cv2.resize(NormBGR, dim, interpolation = cv2.INTER_AREA)
                                        
                                        # Mantengo originales para mostrar parecido
                                        imager = cv2.resize(image, (200, 200), interpolation=cv2.INTER_AREA)
                                        Xorig.append(imager)
                                        # Facenet
                                        X.append(imaged)
                                        Gender.append(sgender)
                                        Age.append(sage)
                                        Etnia.append(setnia)
    
    if nimgs > 0:
        # Compute embeddings 
        embs = calc_embs(np.array(X))

        print("Salvando embs")
        fid = open(outputfile, "wb")

        pickle.dump([nimgs, X, embs, Age, Gender, Etnia], fid)
        print(embs.shape)
        fid.close()


In [ ]:
def GetSimilarNN(X,idxsimilar, nbrs, embs, Title):
    # Gets nearest neighbors
    distance, indices = nbrs.kneighbors(embs)
    minp = indices[0][0]

    # Keeps copy of last grames NN
    idxsimilar.extend(indices[0])

    # Remove old ones when more than nframes accumulated
    if len(idxsimilar) > kvecinos * nframeskvecinos:
        idxsimilar = idxsimilar[kvecinos:]

    # print('Lista con ' + str(len(idxsimilar)) + ' ' + str(idxsimilar) + '\n')

    # enough history
    if len(idxsimilar) > kvecinos:
        minp = GetClosesetMode(idxsimilar)

    # Larger for visualization
    imageS = cv2.resize(X[minp], (320, 320))

    # Soft biometrics labels
    if len(idxsimilar) > 0:
        modesG = stats.mode([Gender[int(i)] for i in idxsimilar])
        modesE = stats.mode([Etnia[int(i)] for i in idxsimilar])
        edad = np.array([int(Age[int(i)]) for i in idxsimilar]).mean()

        if modesG[0][0] == '0':
            gen = 'M'
        else:
            gen = 'F'

        if modesE[0][0] == '0':
            et = 'B'
        elif modesE[0][0] == '1':
            et = 'N'
        elif modesE[0][0] == '2':
            et = 'A'
        elif modesE[0][0] == '3':
            et = 'H'
        else:
            et = 'L'

        cv2.putText(imageS, '%s %s %d (%s)' % (gen, et, int(edad), Age[minp]), (10, 30), font, 0.5, (255, 255, 255), 2,
                    cv2.LINE_AA)

    cv2.imshow(Title, imageS)

    return idxsimilar

def getLargestMTCNNBB(objects):
        if len(objects) < 1:
            return -1
        elif len(objects) == 1:
            return 0
        else:
            areas = [ (det['box'][2]*det['box'][3]) for det in objects ]
            return np.argmax(areas)
        
def DetectLargestFaceEyesMTCNN(img):
    global detectormtcnn
    
    results = detectormtcnn.detect_faces(img)

    if not results is None:
        index = getLargestMTCNNBB(results)

        if len(results) < 1:
            return None

        # laergest face
        face_info = results[index]

        #print(face_info)

        [x, y, w, h] = face_info['box']
        le = face_info['keypoints']['left_eye']
        re = face_info['keypoints']['right_eye']

        return [x,y,w,h], [le[0], le[1], re[0], re[1]], [face_info['keypoints']['left_eye'], face_info['keypoints']['right_eye'],
                      face_info['keypoints']['nose'], face_info['keypoints']['mouth_left'],
                      face_info['keypoints']['mouth_right']]
    else:
        return None
    
def ResetDetectionCounters():
    global nconsecutivenodetected, nconsecutivedetected, idxsimilarFN

    nconsecutivenodetected = nconsecutivenodetected + 1
    if nconsecutivenodetected > 10:
        nconsecutivedetected = 0
        idxsimilarFN = []
        
def GetClosesetMode(list):
    # Get occurrences list
    occ = Counter(list)

    # Get the mode, if multiple modes present, gets the clostest observing position among beigbors
    prima = 0
    maxpun = 0
    for neighbor, count in occ.most_common(10):
        if prima == 0:
            prima = 1
            mode = count
        else:
            if count < mode:
                break

        #print('%s: %7d' % (neighbor, count))

        # using enumerate()
        # to find indices for 3
        neighbor_pos = [i for i, value in enumerate(list) if value == neighbor]
        # printing resultant list
        #print("New indices list : " + str(neighbor_pos))

        pun = 0
        for pos in neighbor_pos:
            pun = pun + kvecinos - (pos % kvecinos)
        #print(pun)
        if pun > maxpun:
            maxpun = pun
            closest_neighbor = neighbor

    return closest_neighbor

# INTERFAZ

In [108]:
import shutil
import time
from tkinter import ttk
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os
from deepface import DeepFace

class EmotionInfoWindow:
    def __init__(self, root, emotion_info, image_path):
        self.root = root
        self.root.title("Información de Emociones")

        # Dividir la ventana en dos secciones
        self.image_frame = tk.Frame(root, width=100, height=200)
        self.info_frame = tk.Frame(root)

        self.image_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.info_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # Mostrar la imagen en la izquierda
        profile_image = Image.open(image_path)
        profile_image = profile_image.resize((150, 150), Image.ADAPTIVE)
        profile_photo = ImageTk.PhotoImage(profile_image)
        self.image_label = tk.Label(self.image_frame, image=profile_photo)
        self.image_label.image = profile_photo
        self.image_label.pack(pady=10)

        # Mostrar la información en la derecha
        self.label_age = ttk.Label(self.info_frame, text=f"Age: {emotion_info['age']}")
        self.label_race = ttk.Label(self.info_frame, text=f"Race: {emotion_info['dominant_race']}")
        self.label_emotion = ttk.Label(self.info_frame, text=f"Emotion: {emotion_info['dominant_emotion']}")
        self.label_gender = ttk.Label(self.info_frame, text=f"Gender: {emotion_info['dominant_gender']}")
        self.label_image_path = ttk.Label(self.info_frame, text=f"Image Path: {image_path}")

        # Ubicar las etiquetas en la ventana
        self.label_age.pack(pady=10)
        self.label_race.pack(pady=10)
        self.label_emotion.pack(pady=10)
        self.label_gender.pack(pady=10)
        self.label_image_path.pack(pady=10)

        # Calcular la posición para colocar la ventana encima de la pantalla principal
        width, height = 400, 200  # Ajusta según sea necesario
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x = root.winfo_x() + (root.winfo_width() - width) // 2
        y = root.winfo_y() - height - 10  # Ajusta según sea necesario
        if y < 0:
            y = 0

        # Establecer la geometría de la ventana
        self.root.geometry(f"{width}x{height}+{int(x)}+{int(y)}")

class PhotoAlbumApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Photo Album App")
        # Contenedor principal
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Contenedor para las imágenes con scrollbar
        self.image_frame = tk.Frame(self.main_frame)
        self.image_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.canvas = tk.Canvas(self.image_frame, width=320, height=480)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar = tk.Scrollbar(self.image_frame, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        # Contenedor interno para las imágenes
        self.image_container = tk.Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.image_container, anchor=tk.NW)

        # Crear un Frame para actuar como separador
        separator_frame = tk.Frame(self.main_frame, width=2, bg="black")
        separator_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10)

        # Contenedor para la información del usuario
        self.user_info_frame = tk.Frame(self.main_frame, width=320, height=480)
        self.user_info_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.label = tk.Label(self.user_info_frame, text="Bienvenido")
        self.label.pack(pady=10)

        # Añadir una etiqueta para la foto de perfil
        self.profile_photo = tk.Label(self.user_info_frame)
        self.profile_photo.pack(pady=10)

        self.data = tk.Label(self.user_info_frame, text="Data: ")
        self.dataInfo = tk.Label(self.user_info_frame, text="")

        #LOG
        self.btn_capture = tk.Button(self.user_info_frame, text="Iniciar Sesion", command=self.iniciar_sesion)
        self.btn_capture.pack(pady=10)

        # Crear el botón para agregar imágenes
        self.btn_add_image = tk.Button(self.user_info_frame, text="Agregar Imagen al Album", command=self.agregar_imagen)
        self.btn_logout = tk.Button(self.user_info_frame, text="Cerrar Sesion", command=self.cerrar_sesion)


        self.canvas.bind("<Button-1>", self.on_image_click)  # Asociar evento de clic a la función
        self.user_folder = "User"
        self.data_folder = "DataAlbum/Galeria/"
        self.profile_folder = "DataAlbum/BaseUsuarios/"
        self.photo_path = ""
        self.photo_references = []  # Lista para almacenar las referencias a las imágenes
        self.user_name = ""


    def iniciar_sesion(self):
        # Efectuar el reconocimiento facial 
        # user_name = self.recognize_user()     RECONOCEMOS CARA O REGISTRAMOS CREADO IMAGEN Y CARPETA EN GALERIA
        user_name = "Acoran"  # Temporal, reemplazar con el reconocimiento facial
        if user_name:
            user_name = user_name.strip()  # Eliminar espacios en blanco al inicio y al final
            self.user_name = user_name

            self.initUser()
            self.btn_add_image.pack(pady=10)
            self.btn_capture.pack_forget()
            self.data.pack(pady=10)
            self.dataInfo.pack(pady=10)
            self.btn_logout.pack(pady=10)
            
            # Abrir la carpeta del usuario y mostrar las imágenes
            user_path = os.path.join(self.data_folder, user_name)
            self.load_and_display_images(user_path)
            self.image_container.update_idletasks()
            self.canvas.config(scrollregion=self.canvas.bbox("all"))

    def initUser(self):
        self.label.config(text=f"Bienvenido, {self.user_name}")

        profile_photo_path = os.path.join(self.profile_folder, f"{self.user_name}.jpg")
        if os.path.exists(profile_photo_path):
            print("existe")
            profile_image = Image.open(profile_photo_path)
            profile_image = profile_image.resize((100, 100), Image.ADAPTIVE)
            profile_photo = ImageTk.PhotoImage(profile_image)
            self.profile_photo.configure(image=profile_photo)
            self.profile_photo.pack(pady=10)
            self.profile_photo.image = profile_photo  # Mantener una referencia para evitar que se elimine la imagen
            obj = DeepFace.analyze(profile_photo_path, enforce_detection=False, actions=['age', 'gender', 'race', 'emotion'])
            self.dataInfo.config(text=f"Name: {self.user_name}\nAge: {obj[0]['age']}\nGender: {obj[0]['dominant_gender']}\nEmotion: {obj[0]['dominant_emotion']} ")



    def cerrar_sesion(self):
            # Restaurar el estado inicial al cerrar la sesión
            self.user_name = ""
            self.photo_references = []
            self.photo_path = ""
            self.label.config(text="Bienvenido")
            self.data.pack_forget()
            self.profile_photo.pack_forget()
            self.dataInfo.pack_forget()
            self.btn_logout.pack_forget()
            self.btn_capture.pack(pady=10)
            self.btn_add_image.pack_forget()
            self.canvas.delete("all")


    def load_and_display_images(self, folder):
        # Cargar las imágenes y almacenar las referencias a PhotoImage
        self.photo_references = []  # Limpiar la lista de referencias a las imágenes

        for filename in os.listdir(folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(folder, filename)
                image = Image.open(img_path)
                image.thumbnail((100, 100))  # Ajustar el tamaño según sea necesario
                photo = ImageTk.PhotoImage(image)

                # Almacenar la referencia a la imagen
                self.photo_references.append((img_path, photo))

        # Mostrar las imágenes en el canvas
        self.display_images()

    def agregar_imagen(self):
        # Abrir un cuadro de diálogo para que el usuario seleccione una imagen
        file_path = filedialog.askopenfilename(title="Seleccionar Imagen", filetypes=[("Imagen", "*.png;*.jpg;*.jpeg")])

        # Verificar si el usuario seleccionó una imagen
        if file_path:
            # Aquí puedes realizar cualquier lógica adicional que necesites con la nueva imagen
            print(f"Imagen seleccionada: {file_path}")

            # Por ejemplo, podrías cargar y mostrar la nueva imagen en el canvas
            image = Image.open(file_path)
            image.thumbnail((100, 100))  # Ajusta el tamaño según sea necesario
            photo = ImageTk.PhotoImage(image)

            # Almacenar la referencia a la nueva imagen
            self.photo_references.append((file_path, photo))

            # Mostrar las imágenes actualizadas en el canvas
            self.display_images()
            
            # Obtener la carpeta del usuario
            user_path = os.path.join(self.data_folder, self.user_name)

            # Copiar la nueva imagen al directorio del usuario
            shutil.copy(file_path, user_path)

    def display_images(self):
        # Código para mostrar las imágenes en el canvas
        # Puedes adaptar este código según tus necesidades
        self.canvas.delete("all")  # Limpiar el canvas

        for i, (photo_path, photo) in enumerate(self.photo_references):
            x_offset = (i % 4) * 120
            y_offset = (i // 4) * 120
            img_id = self.canvas.create_image(x_offset, y_offset, anchor=tk.NW, image=photo)
            self.canvas.tag_bind(img_id, "<Button-1>", lambda event, path=photo_path: self.on_image_click(event, path))

    def on_image_click(self, event, image_path):
        # Función llamada al hacer clic en una imagen
        self.photo_path = image_path
        self.analyze_emotion()

    def analyze_emotion(self):
        # Analizar emociones usando DeepFace
        obj = DeepFace.analyze(self.photo_path, enforce_detection=False, actions=['age', 'gender', 'race', 'emotion'])
        if obj and isinstance(obj, list):
            self.show_emotion_info_window(obj[0], self.photo_path)
        else:
            print("No se encontraron resultados de análisis de emociones.")

    def show_emotion_info_window(self, emotion_info, image_path):
        # Calcular la posición para colocar la ventana encima de la ventana principal
        width, height = 450, 250  # Ajusta según sea necesario
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = self.root.winfo_x() + (self.root.winfo_width() - width) // 2
        y = self.root.winfo_y() + (self.root.winfo_height() - height) // 2

        # Crear una nueva ventana para mostrar la información de emociones
        window = tk.Toplevel(self.root)
        emotion_info_window = EmotionInfoWindow(window, emotion_info, image_path)

        # Establecer la geometría de la nueva ventana
        window.geometry(f"{width}x{height}+{int(x)}+{int(y)}")



if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoAlbumApp(root)
    root.mainloop()


existe


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  7.12it/s]
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1776.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: PhotoAlbumApp.on_image_click() missing 1 required positional argument: 'image_path'
Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  7.63it/s]
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1776.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: PhotoAlbumApp.on_image_click() missing 1 required positional argument: 'image_path'


Imagen seleccionada: C:/Users/acora/OneDrive/Imágenes/borrar.jpg


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  7.71it/s]
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1776.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: PhotoAlbumApp.on_image_click() missing 1 required positional argument: 'image_path'
